In [27]:
# Zelle 1: Imports und Basis-Setup

from groq import Groq
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDFS, RDF, XSD
import json
from textwrap import dedent

from groq import Groq

# Pfad zu deiner Datei
api_key_path = r"C:\Users\Alexander Verkhov\Desktop\APIKey_Groq.txt"

# API-Key aus Datei lesen
with open(api_key_path, "r", encoding="utf-8") as f:
    groq_api_key = f.read().strip()  # .strip() entfernt Zeilenumbrüche

# Client mit Key initialisieren
client = Groq(api_key=groq_api_key)

# Testaufruf
resp = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Hallo, sprichst du auch deutsch?"}],
    max_completion_tokens=500,
)

print(resp.choices[0].message.content)

def call_groq(messages, model="llama-3.1-8b-instant", temperature=0.0, max_tokens=1024):
    """
    Einfacher Wrapper für Groq-ChatCompletion (nicht-streamend).
    messages: Liste von {role: "system"/"user"/"assistant", content: "..."}
    Gibt IMMER einen JSON-String zurück (JSON Object Mode aktiviert).
    """
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_completion_tokens=max_tokens,
        top_p=1,
        stream=False,
        # WICHTIG: JSON Object Mode erzwingt gültiges JSON
        response_format={"type": "json_object"},
    )
    content = resp.choices[0].message.content
    # Sicherheitshalber trimmen
    return content.strip() if content is not None else ""


Ja, ich spreche Deutsch. Wie kann ich dir helfen?


In [28]:

# Pfad zu deinem Graphen (anpassen, falls anders)
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent_filled2.ttl"

g = Graph()
g.parse(TTL_PATH, format="turtle")
print(f"Graph geladen mit {len(g)} Tripeln.")

# Namespaces (so wie in deinem Beispiel)
AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")
OP = Namespace("http://www.semanticweb.org/AgentProgramParams/op_")
DP = Namespace("http://www.semanticweb.org/AgentProgramParams/dp_")

g.bind("ag", AG)
g.bind("op", OP)
g.bind("dp", DP)

Graph geladen mit 4378 Tripeln.


In [29]:
# %%
def get_local_name(uri: str) -> str:
    """
    Gibt den lokalen Namen einer URI zurück (alles nach dem letzten '#' oder '/').
    """
    if "#" in uri:
        return uri.rsplit("#", 1)[-1]
    return uri.rstrip("/").rsplit("/", 1)[-1]


def get_all_program_uris(graph: Graph):
    """
    Liefert alle Individuen vom Typ ag:class_Program.
    """
    return sorted(
        {str(p) for p in graph.subjects(RDF.type, AG.class_Program)}
    )


def run_var_query_for_program(graph: Graph, program_uri: str):
    """
    Führt pro Programm eine SPARQL-Abfrage aus, ähnlich deiner varquery.
    Gibt strukturierte Daten zurück:
      - code: str
      - inputs / outputs / internals / used: Listen von URIs (Strings)
    """
    # Program URI für SPARQL
    program_sparql = f"<{program_uri}>"

    varquery = f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX ag:<http://www.semanticweb.org/AgentProgramParams/>
    PREFIX op: <http://www.semanticweb.org/AgentProgramParams/op_>
    PREFIX dp: <http://www.semanticweb.org/AgentProgramParams/dp_>
    SELECT ?program ?ivar ?ovar ?intvar ?usedvar ?code
    WHERE {{
        VALUES ?program {{ {program_sparql} }}
        OPTIONAL {{ ?program op:hasInputVariable ?ivar . }}
        OPTIONAL {{ ?program op:hasOutputVariable ?ovar . }}
        OPTIONAL {{ ?program op:hasInternalVariable ?intvar . }}
        OPTIONAL {{ ?program op:usesVariable ?usedvar . }}
        ?program dp:hasProgramCode ?code .
    }}
    """

    results = graph.query(varquery)

    inputs = set()
    outputs = set()
    internals = set()
    usedvars = set()
    code_str = None

    for row in results:
        if row.ivar:
            inputs.add(str(row.ivar))
        if row.ovar:
            outputs.add(str(row.ovar))
        if row.intvar:
            internals.add(str(row.intvar))
        if row.usedvar:
            usedvars.add(str(row.usedvar))
        if row.code:
            # Falls mehrere Zeilen mit gleichem Code kommen, einfach den ersten nehmen
            if code_str is None:
                code_str = str(row.code)

    return {
        "program_uri": program_uri,
        "code": code_str,
        "inputs": sorted(inputs),
        "outputs": sorted(outputs),
        "internals": sorted(internals),
        "usedvars": sorted(usedvars),
    }


In [30]:
import json
import re

def analyze_program_with_llm(program_uri, program_code, kg_variables):
    """
    Ruft das LLM auf, um Variablen im Programcode mit den KG-Variablen
    (inputs, outputs, internals, used) abzugleichen.

    Erwartetes JSON vom LLM (JSON Object Mode):
    {
      "program_uri": "...",
      "inputs_from_code": [...],
      "outputs_from_code": [...],
      "internals_from_code": [...],
      "per_variable_report": [
        {
          "kg_uri": "...",
          "kg_role": "input|output|internal|used",
          "kg_name": "Var_Foo",
          "matching_code_variables": ["Foo"],
          "present_in_code": true/false,
          "comment": "..."
        }
      ]
    }
    """
    payload = {
        "program_uri": program_uri,
        "program_code": program_code,
        "kg_variables": kg_variables,
    }

    system_prompt = (
        "Du bist ein sehr genauer Assistent für Code-Analyse. "
        "Du bekommst den Code eines Programms (Python-Übersetzung aus PLC), "
        "sowie eine Liste von Variablen aus einem Wissensgraphen.\n\n"
        "Aufgabe:\n"
        "1. Leite aus dem Programcode ab, welche Variablennamen als Inputs, Outputs "
        "   und interne Variablen auftreten. Ignoriere Namen, die mit 'V_' anfangen, "
        "   weil das Funktionsbaustein-Zwischenwerte sind.\n"
        "2. Vergleiche für JEDES Element in kg_variables, ob der zugehörige Name (ohne 'Var_'-Präfix) "
        "   irgendwo im Programcode vorkommt.\n"
        "3. Erzeuge für jede KG-Variable einen Eintrag in 'per_variable_report' "
        "   mit:\n"
        "   - kg_uri\n"
        "   - kg_role (input / output / internal / used)\n"
        "   - kg_name (z.B. 'Var_ColorCode_1')\n"
        "   - matching_code_variables (Liste von Code-Variablennamen, die dazu passen)\n"
        "   - present_in_code (true/false)\n"
        "   - comment (kurzer deutscher Kommentar, z.B. 'Name im KG und im Code konsistent (Input)' "
        "              oder 'Nur als Input im KG modelliert, im Code nicht gefunden.')\n\n"
        "Antworte AUSSCHLIESSLICH mit einem einzigen JSON-Objekt in genau diesem Format. "
        "KEINE Erklärungen, KEINE Codeblöcke, KEINE zusätzlichen Felder."
    )

    user_prompt = json.dumps(payload, ensure_ascii=False, indent=2)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    raw_response = call_groq(messages, max_tokens=1024)

    # Da JSON Object Mode aktiv ist, sollte das hier direkt gültiges JSON sein.
    try:
        result = json.loads(raw_response)
    except json.JSONDecodeError as e:
        print(f"Fehler beim JSON-Parsing für Programm {program_uri}: {e}")
        print("Rohantwort des LLM:")
        print(raw_response)
        return None

    # Minimalprüfung auf erwartete Struktur
    if not isinstance(result, dict) or "per_variable_report" not in result:
        print(f"Unerwartete Struktur vom LLM für Programm {program_uri}.")
        print(result)
        return None

    return result


In [31]:
def build_summary_text(program_uri: str, analysis: dict) -> str:
    """
    Baut einen deutschen Klartext-Bericht aus per_variable_report.
    Jede KG-Variable bekommt eine Zeile.
    """
    lines = []
    per_var = analysis.get("per_variable_report", [])

    for entry in per_var:
        kg_name = entry.get("kg_name", "?")
        role = entry.get("kg_role", "?")
        present = entry.get("present_in_code", False)
        comment = entry.get("comment", "")
        present_str = "im Code vorhanden" if present else "im Code NICHT vorhanden"
        line = f"{kg_name} ({role}): {present_str}. {comment}"
        lines.append(line)

    lines.append(f"Gesamtfazit: {program_uri} hat {len(per_var)} modellierte Variablen im Bericht.")

    return "\n".join(lines)


In [ ]:
def build_kg_variables_from_info(info: dict) -> dict:
    """
    Baut aus dem Ergebnis von run_var_query_for_program(...) die Struktur,
    die analyze_program_with_llm erwartet:

    {
      "inputs":   [ { "uri": ..., "kg_name": ..., "role": "input"   }, ... ],
      "outputs":  [ { "uri": ..., "kg_name": ..., "role": "output"  }, ... ],
      "internals":[ { "uri": ..., "kg_name": ..., "role": "internal"}, ... ],
      "usedvars": [ { "uri": ..., "kg_name": ..., "role": "used"    }, ... ]
    }
    """

    def mk_list(key: str, role: str):
        out = []
        for uri in info.get(key, []):
            local_name = get_local_name(uri)  # get_local_name hast du ja schon definiert
            out.append(
                {
                    "uri": uri,
                    "kg_name": local_name,
                    "role": role,
                }
            )
        return out

    return {
        "inputs":    mk_list("inputs",    "input"),
        "outputs":   mk_list("outputs",   "output"),
        "internals": mk_list("internals", "internal"),
        "usedvars":  mk_list("usedvars",  "used"),
    }

def add_inconsistency_annotations(graph: Graph, limit_programs: int | None = None):
    program_uris = get_all_program_uris(graph)
    print(f"Gefundene Programme: {len(program_uris)}")

    if limit_programs is not None:
        program_uris = program_uris[:limit_programs]
        print(f"Analysiere nur die ersten {len(program_uris)} Programme.")

    for i, p_uri in enumerate(program_uris, start=1):
        print(f"\n[{i}/{len(program_uris)}] Analysiere Programm: {p_uri}")
        info = run_var_query_for_program(graph, p_uri)

        if not info.get("code"):
            print("  -> Übersprungen (kein Code gefunden).")
            continue

        # NEU: kg_variables aus info bauen
        kg_variables = build_kg_variables_from_info(info)

        # NEU: analyze_program_with_llm mit 3 Argumenten aufrufen
        analysis = analyze_program_with_llm(
            program_uri=info["program_uri"],
            program_code=info["code"],
            kg_variables=kg_variables,
        )

        if not analysis:
            print("  -> Keine verwertbare LLM-Antwort, übersprungen.")
            continue

        # summary_text jetzt in Python erzeugen (falls du build_summary_text schon hast)
        summary_text = build_summary_text(p_uri, analysis)

        program_ref = URIRef(p_uri)
        lit = Literal(summary_text, datatype=XSD.string)

        graph.add((program_ref, DP.hasConsistencyReport, lit))

        print("  -> dp:hasConsistencyReport hinzugefügt.")

    print("\nFertig.")

In [38]:
add_inconsistency_annotations(g, limit_programs=50)

# %%
OUTPUT_TTL = r"D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent_filled2.ttl"
g.serialize(OUTPUT_TTL, format="turtle")
print(f"Graph mit Inconsistency-Kommentaren gespeichert als: {OUTPUT_TTL}")

Gefundene Programme: 50
Analysiere nur die ersten 50 Programme.

[1/50] Analysiere Programm: http://www.semanticweb.org/AgentProgramParams/Program_AutomaticColorDetection_nichtfertig
  -> dp:hasInconsistentTranslation hinzugefügt.

[2/50] Analysiere Programm: http://www.semanticweb.org/AgentProgramParams/Program_AxisControl_Encoder
  -> dp:hasInconsistentTranslation hinzugefügt.

[3/50] Analysiere Programm: http://www.semanticweb.org/AgentProgramParams/Program_AxisControl_MultipleInputSensors
  -> dp:hasInconsistentTranslation hinzugefügt.

[4/50] Analysiere Programm: http://www.semanticweb.org/AgentProgramParams/Program_CompressorControl


APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01kanyhbg2ecwsvevy4q8vapa7` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 9559, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}